# Factory Method

The Factory Method is a **Creational Design Pattern** which aims to encapsulate the creation of objects and to facilitate its reusability.

In this design pattern, instead of directly instanciating a class, the main code asks to a factory function for an object without knowning *how* it is created or from *where*; It does not care about the class that generates it nor about how it is implemented. 

But... **WHEN** should we use the Factory Method instead of direct class instanciation?
- If you cannot track easily all the objects created in your app because **the code that creates them is in many different places**, you should consider this design pattern. The Factory Method **centralizes** object creation so that any change you would like to do in the way objects are created would have to be modified **only once**: in the factory method. 
- If you want to **decouple object creation from its usage** so that adding new objects or modifying the way of creating them would only affect to the creation part, not to the code which uses them. 
- If you don't know all the possible objects you have to generate at a time, or your app is open up to new object types in the future.
- In addition, the Factory Method shows better performance and memory usage than using direct class instantiation. So you should consider it if you are working with large objects and you want to minimize resources usage and keep your app efficient. 

The Factory Method has some advantages as well as disadvantages. The main advantages are:
- You decople object creation from its concrete implementations.
- It's easier to satisfy the Single Reponsibility Principle (SRP) (SOLID).
- It's easier to satisfy the Open Closed Principle (OCP) (SOLID).

However:
- The code becomes more complex and you will have to create new multiple subclasses. 

Using the Factory Method we have four maining classes:
- First, we have a *Creator* class. This is an abstract class and the parent of all the factories used in the system. At least, it has one method called the *factoryMethod()* which is the responsible of creating and returning new objects. Often, these objects are called *Products*. Optionally, the *factoryMethod()* can receive some parameters.
- From the *Creator* class, we inherit all the *ConcreteCrator* classes, whose aim is to create new *ConcreteProducts*. Each *ConcreteCreator* overrides and implements the *factoryMethod()* derived from the *Creator* class. In other words, *Creator* classes are the factories of our products.
- On the other side, we have a *Product* abstract class, which is the base class for all the existing concrete products on our app.
- Finally, we have one or more *ConcreteProduct* classes, which are classes derived from *Product* that represent specific products on our app. 

According to these parts, it's completely OK to have multiple factories methods. Each factory method logically groups the creation of objects that have some similarities. 

Here is the UML diagram of the Factory Method pattern:

![](UML-Diagrams/FactoryMethod-ClassDiagram.png)

And here is the Python code of the Factory Method pattern:

In [3]:
from abc import ABC, abstractmethod


class Product(ABC):
    """Abstract class that represent a product."""


class ConcreteProduct(Product):
    """A specific product."""


class Creator(ABC):
    """Abstract class that represents a factory or creator."""

    @abstractmethod
    def factory_method(self) -> Product:
        """Creates a Product."""


class ConcreteCreator(Creator):
    """A concrete factory or creator."""

    def _factory_method(self) -> ConcreteProduct:
        """Create a ConcreteProduct and returns it."""
        return ConcreteProduct()


## Example: 

We are going to develop a parser that reads input files with multiple formats such as XML and JSON. 

We are going to use the Factory Method to create the connectors or readers of each file format. For example, we are going to have one class to read XML ('XMLConnector') and another one to read JSON ('JSONConnector'). These are the *ConcreteProducts* of the pattern. We will also have another abstract class called 'Connector' that defines a property called 'parsed_data()'. 'Connector' is the so-called *Product* class of the pattern. 

Finally, we will have a 'ConnectionFactory', which is the *Creator* of the pattern, that defines a factory method called 'create_connection()' that returns a 'Connector', that is, a *Product*. From 'ConnectionFactory', we will inherit a class called 'ConcreteConnectionFactory' that implements the 'create_connection()' method and returns specific types of connections, such as 'XLMConnector' or 'JSONConnector', that is, *ConcreteProduct*s.

This info is represented in the following UML diagram:

![](UML-Diagrams/ExampleFactoryMethod.png)

And here's the code:

In [29]:
# First, let's define our products
from abc import ABC, abstractmethod
import json
import xml.etree.ElementTree as ET

# We create the parent abstract class for all our connectors.
class Connector(ABC):
    @abstractmethod
    def __init__(self, filepath: str) -> None:
        """Constructor."""

    @property
    def parsed_data(self) -> str:
        """A property that returns the parsed data."""


# And now we inheritc from Connector all the existing connectors in our app
class JSONConnector(Connector):
    """A JSON reader."""

    def __init__(self, filepath: str) -> None:
        self._data = dict()
        try:
            with open(filepath, "r", encoding="utf-8") as f:
                self._data = json.load(f)
        except Exception as e:
            raise e

    @property
    def parsed_data(self) -> str:
        return json.dumps(self._data)


class XMLConnector(Connector):
    def __init__(self, filepath: str) -> None:
        tree = ET.parse(filepath)
        self._root = tree.getroot()

    @property
    def parsed_data(self) -> str:
        return ET.tostring(self._root, encoding="utf-8").decode("utf-8")


In [14]:
# Now let's implement the Factory Pattern

# Let's define the creators:
class ConnectionFactory(ABC):
    @abstractmethod
    def create_connection(self, filepath: str) -> Connector:
        """Creates a connector."""


class ConcreteConnectionFactory(ConnectionFactory):
    def create_connection(self, filepath: str) -> Connector:
        if filepath.endswith("json"):
            return JSONConnector(filepath)
        elif filepath.endswith("xml"):
            return XMLConnector(filepath)
        else:
            raise NotImplementedError(
                f"Unsuported filetype for file: {filepath}."
            )


In [32]:
# Now let's used the factory with JSON:

if __name__ == "__main__":
    
    # You can change here the file you want to read
    input_filepath = r"./data_samples/example.json"
    #input_filepath = r"./data_samples/example.xml"
    
    factory = ConcreteConnectionFactory()
    reader = factory.create_connection(input_filepath)
    print(reader.parsed_data)
    

[{"name": "Anna", "age": 26, "country": "Spain"}, {"name": "Isabella", "age": 25, "country": "Italy"}]


In [33]:
# Or with XML. Notice that this code remains the same (except the filepath)

if __name__ == "__main__":

    # You can change here the file you want to read
    # input_filepath = r"./data_samples/example.json"
    input_filepath = r"./data_samples/example.xml"

    factory = ConcreteConnectionFactory()
    reader = factory.create_connection(input_filepath)
    print(reader.parsed_data)


<persons>
    <person>
        <name>Anna</name>
        <age>26</age>
        <country>Spain</country>
    </person>
    <person>
        <name>Anna</name>
        <age>26</age>
        <country>Spain</country>
    </person>
</persons>


*Sources*:
- Mastering Python Design Patterns (Sakis Kasampalis)
- YouTube Video: https://youtu.be/lLvYAzXO7Ek 